In [ ]:
## importamos las librerías necesarias
import requests ## permite realizar solicitudes HTTP
from bs4 import BeautifulSoup ## crea la "sopa", es decir, extrae el código de HTML de la página para poder obtener los datos 
import re ## busca patrones de texto
import pandas as pd ##crea y manipula Data Frames

In [ ]:
## El siguiente código accede a la página del ranking FIFA del 9 de enero de 2023 y obtiene el nombre, enlace de perfil, enlace
## del calendario de los primeros 25 equipos nacionales. Al final, guarda toda esa información en un diccionario y le agrega el
## año de análisis.

page= 'https://www.transfermarkt.mx/statistik/weltrangliste/statistik/stat/plus/0/galerie/0?datum=2023-01-09'
headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
page_tree = requests.get(page, headers=headers)
soup = BeautifulSoup(page_tree.content, 'html.parser')
table= soup.find('table', {'class':'items'})
table_body= table.find('tbody')
tds= table_body.find_all('td', {'class':'hauptlink'})
seleccionlink={'Selección':[],'Link':[],'ScheLink':[], 'Year':[]}
linksolo=[]
linkfinal=[]
tdsnecesarios=[]
seleccion=[]
for y in range(0, len(tds), 2):
    tdsnecesarios.append(y)
for j, te in enumerate(tds):
    for sele in range(0, len(tdsnecesarios)):
        if j==tdsnecesarios[sele]:
            img=te.find('img')
            a=te.find('a')
            seleccion.append(img['alt'])
            linksolo.append(a['href'])
for i in range(0,len(linksolo)):
    lin=linksolo[i].replace('startseite', 'spielplan')
    linkfinal.append(lin)
for l in range(0,len(seleccion)):
    for year in range(2008, 2023):
            y = str(year)
            seleccionlink['Selección'].append(seleccion[l])
            seleccionlink['Link'].append('https://www.transfermarkt.mx' + linksolo[l] + '?saison_id=' + y)
            seleccionlink['Year'].append(y)
            seleccionlink['ScheLink'].append('https://www.transfermarkt.mx' + linkfinal[l] + '/saison_id/' + y)

In [ ]:
## El siguiente código utiliza la información que obtuvimos en la celda anterior para obtener los minutos jugados por el equipo
## nacional en un año específico.

minutossele={'NationalTeam':[], 'Year':[], 'Year (T-1)':[],'Minutes':[]}
for i in range(0, len(seleccionlink['Selección'])):    
    selección = seleccionlink['Selección'][i]
    añoconv = int(seleccionlink['Year'][i])+1
    añomins = int(seleccionlink['Year'][i])
    page = seleccionlink['ScheLink'][i]
    headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
    page_tree = requests.get(page, headers=headers)
    soup = BeautifulSoup(page_tree.content, 'html.parser')
    table = soup.find_all('div', {'class': 'responsive-table'})
    tablas=[]
    for row in table:
        tabs=row.find('tbody')
        tablas.append(tabs)
    marcadores=[]
    for row in tablas:
        marcador=row.find_all('a',{'title':'Crónica'})
        for row in marcador:
            no= row.text
            marcadores.append(no)
    minutos=[]
    for i in range(0,len(marcadores)):
        extra=marcadores[i]
        if 'pen.' in extra:
            minutos.append(int(120))
        elif 'prórr.' in extra:
            minutos.append(int(120))
        else:
            minutos.append(int(90))
    minutossele['NationalTeam'].append(selección)
    minutossele['Year'].append(añoconv)
    minutossele['Year (T-1)'].append(añomins)
    minutossele['Minutes'].append(sum(minutos))


In [ ]:
## Esta celda "arregla" los años de análisis
for i in range(0, len(minutossele['Minutes'])):
    minutossele['Year'][i]=minutossele['Year'][i]+1
    minutossele['Year (T-1)'][i]=minutossele['Year (T-1)'][i]+1

In [ ]:
minutos=pd.DataFrame(minutossele)

In [ ]:
minutos.to_excel('Nombre.xlsx', encoding='utf')